**Sakshi Patil **
<br> B 2018130039

##Experiment 3

**Objective:**

(i) Apply Naive bayes classifier for given data see Naive_bayes_handout.zip file in edmodo.

(ii) Apply and compare Linear Discriminating Analysis with Naive bayes classifiers w.r.t. error, specificity and sensitivity

Load the construct an python code for:

1. Convert the textual meta-data into a suitable (e.g. corpus) object.

2. Triage some of the irrelevant punctuation and other symbols in the corpus document,change all text to lower case, etc.

3. Tokenize the job descriptions into words. Examine the distributions of two important features

4. Classify attributes in two categories.

5. Graphically visualize the difference between low and high value of targeted feature graph.

6. Transform the features into categorical data

7. Ignore those low frequency words and report the sparsity of your categorical data matrix with or without delete those low frequency words. Note that the sparsity of a matrix is the fraction:

Sparsity(A) = number of zero-valued elements / total number of matrix elements (m×n).

1. Apply the Naive Bayes classifier to original matrix and lower dimension matrix, what do you observe?

2. Apply and compare LDA and Naive Bayes classifiers with respect to the error, specificity and sensitivity.

##Naive Bayes Classifier

In [ ]:
!pip install kaggle
!pip install scikit-plot

In [ ]:
!unzip "/content/drive/MyDrive/DatasetS/IMDB/IMDB Dataset.csv.zip"

unzip:  cannot find or open /content/drive/MyDrive/DatasetS/IMDB/IMDB Dataset.csv.zip, /content/drive/MyDrive/DatasetS/IMDB/IMDB Dataset.csv.zip.zip or /content/drive/MyDrive/DatasetS/IMDB/IMDB Dataset.csv.zip.ZIP.


In [ ]:
import numpy as np
import pandas as pd
import scikitplot as skplt

import nltk
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install pyspellchecker 
from spellchecker import SpellChecker

     |████████████████████████████████| 2.7 MB 7.0 MB/s 


In [ ]:
tweets_df = pd.read_csv('/content/Corona_NLP_train.csv', encoding='latin-1')
imdb_data = pd.read_csv("/content/IMDB Dataset.csv")

tweets_df = tweets_df[['OriginalTweet', 'Sentiment']]

test_df = pd.read_csv('/content/Corona_NLP_test.csv')
test_df = test_df[['OriginalTweet', 'Sentiment']]

In [ ]:
# Check for null values
for column in tweets_df.columns:
    print(tweets_df[column].isnull().value_counts())
print()  
for column in test_df.columns:
    print(test_df[column].isnull().value_counts())

False    41157
Name: OriginalTweet, dtype: int64
False    41157
Name: Sentiment, dtype: int64

False    3798
Name: OriginalTweet, dtype: int64
False    3798
Name: Sentiment, dtype: int64


In [ ]:
# Check for duplicate values
print(tweets_df.duplicated().value_counts())
print(test_df.duplicated().value_counts())

False    41157
dtype: int64
False    3798
dtype: int64


In [ ]:
nltk.download('stopwords')
#ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
spell = SpellChecker()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
puncs_ = string.punctuation.replace('@','')
puncs = puncs_.replace('#','')
puncs

'!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [ ]:
def textClean(text):
    # convert to lowercase
    lower = [char.lower() for char in text if char not in puncs]
    lower = ''.join(lower)
    lower = ' '.join(lower.split())
    
    # delete @mentions and #tags
    for char in lower:
        if lower.find('@')==-1 and lower.find('#')==-1: # break loop once @ and # is over
            break
        if (char=='@' or char=='#'):
            try:
                char_index = lower.index(char)
            except ValueError:
                #print(lower)
                break
            del_word = ''
            while char not in string.whitespace:
                del_word = del_word+lower[char_index]
                char_index = char_index + 1
                try:
                    char = lower[char_index] #trying for indexerror incase it is the last character of string
                except IndexError:
                    char = ' '
                except:
                    print("Something else went wrong")
            lower = lower.replace(del_word,'',1)
    lower = [char for char in lower if char not in string.punctuation and char not in string.digits]
    lower = ''.join(lower)
    
    tokens = word_tokenize(lower)
    nohttp = [word for word in tokens if word[0:4]!='http']
    nostop = [word for word in nohttp if word not in stopwords.words('english')]
    return nostop

In [ ]:
temp_list = tweets_df.OriginalTweet[0:10].apply(textClean)
for each_list in temp_list:
    print(each_list)#for word in each_list:

[]
['advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'online', 'shopping', 'accounts', 'poss', 'adequate', 'supplies', 'regular', 'meds', 'order']
['coronavirus', 'australia', 'woolworths', 'give', 'elderly', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'covid', 'outbreak']
['food', 'stock', 'one', 'empty', 'please', 'dont', 'panic', 'enough', 'food', 'everyone', 'take', 'need', 'stay', 'calm', 'stay', 'safe']
['ready', 'go', 'supermarket', 'outbreak', 'im', 'paranoid', 'food', 'stock', 'litteraly', 'empty', 'serious', 'thing', 'please', 'dont', 'panic', 'causes', 'shortage']
['news', 'regionâ\x92s', 'first', 'confirmed', 'covid', 'case', 'came', 'sullivan', 'county', 'last', 'week', 'people', 'flocked', 'area', 'stores', 'purchase', 'cleaning', 'supplies', 'hand', 'sanitizer', 'food', 'toilet', 'paper', 'goods', 'reports']
['cashier', 'grocery

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=textClean)
message = vectorizer.fit_transform(tweets_df['OriginalTweet'])
message.shape

(41157, 39097)

In [ ]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(message,tweets_df.Sentiment,test_size=0.20,random_state=123)

In [ ]:
# create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(xtrain, ytrain)

In [ ]:
print(classifier.predict(xtest))
print(ytest.values)

['Negative' 'Positive' 'Positive' ... 'Neutral' 'Extremely Negative'
 'Positive']
['Negative' 'Positive' 'Neutral' ... 'Neutral' 'Extremely Negative'
 'Positive']


In [ ]:
# Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtrain)
print(classification_report(ytrain, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytrain, pred))
print("Accuracy: \n", accuracy_score(ytrain, pred))

                    precision    recall  f1-score   support

Extremely Negative       0.89      0.65      0.75      4360
Extremely Positive       0.83      0.71      0.77      5273
          Negative       0.69      0.79      0.74      7970
           Neutral       0.93      0.55      0.69      6206
          Positive       0.63      0.86      0.73      9116

          accuracy                           0.73     32925
         macro avg       0.79      0.71      0.73     32925
      weighted avg       0.77      0.73      0.73     32925


Confusion Matrix: 
 [[2823   28 1084   39  386]
 [  17 3761  196   16 1283]
 [ 221  145 6304   91 1209]
 [  51  138  917 3383 1717]
 [  71  479  643   92 7831]]
Accuracy: 
 0.7320273348519363


In [ ]:
# Evaluating the model on the testing data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtest)
print(classification_report(ytest, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytest, pred))
print("Accuracy: \n", accuracy_score(ytest, pred))

                    precision    recall  f1-score   support

Extremely Negative       0.62      0.38      0.47      1121
Extremely Positive       0.60      0.44      0.51      1351
          Negative       0.42      0.51      0.46      1947
           Neutral       0.65      0.32      0.43      1507
          Positive       0.42      0.63      0.50      2306

          accuracy                           0.48      8232
         macro avg       0.54      0.45      0.47      8232
      weighted avg       0.52      0.48      0.48      8232


Confusion Matrix: 
 [[ 427    9  536   20  129]
 [  10  591   77   24  649]
 [ 192   63  989   99  604]
 [  25   45  336  479  622]
 [  33  274  432  115 1452]]
Accuracy: 
 0.478377065111759


##Conclusion:

Through this experiment i performed naive bayes classifier and it's comparison with linear discriminating analysis wrt sensitivity and specificity
The comparison of LDA and Naive Bayes based on various metrics such as Specificity and Sensitivity was performed. The values for this metrics were observed to be increased after reducing the dimensions.
Dimensionality Reduction reduces the time and storage space required. It helps remove multi-collinearity which improves the interpretation of the parameters of the machine learning model. It becomes easier to visualize the data when reduced to very low dimensions such as 2D or 3D.